## import libs

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils import data as torchData
import torch.nn.functional as F
import os
import csv
import pickle
import numpy as np
import time
import util
import gc
import librosa
import soundfile as sf

## Setting Hyperparams 

In [9]:
sequence_len = 101
input_size = 10
hidden_size= 257
num_layers = 2
num_classes = 1

num_epochs = 30
batch_size = 1
learning_rate = 0.001

In [10]:
inPath = "./dataset/accel_250"
inPath2 = "./dataset/spectrogram"
outPath = "./output_model/"

## Data Load

In [26]:
class AudioLoader(torchData.Dataset):

    def __init__(self, inPathAccel, inPathAudio,isShuffle=False):

        files_accel = os.listdir(inPathAccel) 
        files_accel = [f for f in files_accel if os.path.splitext(f)[-1] == '.csv']
        files_accel.sort()
        files_audio = os.listdir(inPathAudio) 
        files_audio = [f for f in files_audio if os.path.splitext(f)[-1] == '.pickle']
        files_audio.sort()

        if isShuffle:
            random.shuffle(files_accel)

        self.inPathAccel = inPathAccel
        self.inPathAudio = inPathAudio
        self.len = len(files_accel)
        self.fileList_accel = files_accel[:self.len]
        self.fileList_audio = files_audio[:self.len]
        self.isShuffle = isShuffle

        print('len of dataset : ',self.len, len(files_audio))


    def __getitem__(self, idx):

        with open(os.path.join(self.inPathAccel , self.fileList_accel[idx]),'r',encoding='utf-8') as csvfile:
    
            rdr = csv.reader(csvfile)
            data_accel = [line for line in rdr]

            for idx2,each_line in enumerate(data_accel) :

                each_line = [float(i) for i in each_line]

                #x,y,z 3 axis -> sum(x,y,z) 1 axis and material property
                sum_3axis = np.sum(each_line[0:2])
                sum_3axis = (sum_3axis-9) / 4
                each_line = sum_3axis#[sum_3axis, each_line[-1]]

                data_accel[idx2] = each_line

            output_data = list()
            result = list()

            pointer = 0
            pointer_bool = True
            for i in range(0,sequence_len):

                if pointer < 9:#input_size-1
                    output_data = data_accel[:pointer+1]
                    output_data = np.pad(output_data, (input_size-pointer-1,0),'constant',constant_values=(0))
                else:
                    output_data = data_accel[pointer-9:pointer+1]

                if pointer_bool :
                    pointer += 2
                else:
                    pointer +=3

                pointer_bool =  not pointer_bool

                if i == sequence_len-1:
                    result.append(result[-1])
                else:
                    result.append(output_data)

            result = np.array(result)
            result = torch.from_numpy(result)

                #output_data = np.array(output_data)
                #print(output_data)
                #output_data = torch.from_numpy(output_data)

            #with open(os.path.join(self.inPathAudio, self.fileList_audio[idx]),'rb') as fs:

            with open(os.path.join(self.inPathAudio+'/'+self.fileList_audio[idx]),'rb') as fs:
                spectro = pickle.load(fs)
                #print(self.inPathAudio+'/'+self.fileList_audio[idx])
                spectro = np.array(spectro)
                spectro = np.log1p(spectro)
                spectro /= 2.2
                label = torch.from_numpy(spectro)
            return result, label #data_audio #input-label

    def __len__(self):
        return self.len


In [27]:
dataSet = AudioLoader(os.path.join(inPath),os.path.join(inPath2))

trainLoader = torchData.DataLoader(
    dataset = dataSet,
    batch_size = batch_size,
    shuffle = False
)

len of dataset :  201 201


# Architecture

In [28]:
class Haptic2AudioRNN(nn.Module):
    def __init__(self):
        super(Haptic2AudioRNN,self).__init__()
        self.model = nn.RNN(input_size =input_size, hidden_size = hidden_size,
                             num_layers = num_layers, batch_first = True, nonlinearity = 'relu')

    def forward(self, x, hidden):       
        
        x = x.view(batch_size, sequence_len, input_size)
        out, hidden = self.model(x, hidden)
        return hidden, out.view(-1, num_classes)

    def init_hidden(self):
        # Set initial states 
        return Variable(torch.zeros(num_layers,batch_size, hidden_size).type(torch.DoubleTensor))

    
def save(outPath, prefix = ''):

    timeText = util.getTime()

    if not prefix == '':

        prefix = prefix + '_'

    try:
        torch.save(self.model.cpu(), os.path.join(outPath, timeText + prefix+'rnn.model'))
    except:
        print('error : can\'t save model')

    else:
        print('successfully saved model - ', timeText + prefix)
        
        
def load(inPath, time = '', num = 1):

    try:
        #load the model files which are created lastly
        files = os.listdir(inPath)
        files = [f for f in files if os.path.splitext(f)[-1] == '.model']
        #files.sort(reverse = True)
        files.sort()
        #timeText = files[0][:10] + '_'
        #self.model = torch.load(os.path.join(inPath, timeText + prefix + 'rnn.model'))
        model = torch.load(os.path.join(inPath, files[-num]))
        #if torch.cuda.is_available():
        #    self.model.cuda()

    except:
        print('error : can\'t load model')

    else:
        #print('successfully loaded all model - ',timeText + prefix)
        print('successfully loaded all model - ',files[-num])
        

In [29]:
model = Haptic2AudioRNN()
model.double()
#def criterion(y, outputs):
#    y = 
criterion1 = nn.KLDivLoss(size_average=True)#size_average = True
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:

print('Train Start...')

lossHistory = list()

hidden= None
hidden = model.init_hidden()

for epoch in range(num_epochs):

    start_time = time.time()
    
    for idx, data in enumerate(trainLoader):#,0

        x, y = data
        x = Variable(x)

        y = Variable(y.type(torch.DoubleTensor), requires_grad = False)

        optimizer.zero_grad()
        loss = 0
        
        hidden, outputs = model.forward(x,hidden)
        #print(hidden)
        #print(outputs)
        #print("- outputs")
        print(torch.mean(hidden))
        outputs = outputs.view(101,257)

        '''
        print("- y")
        util.printInfo(y)
        util.printInfo(F.softmax(y.view(101*257)))
        util.printInfo(F.log_softmax(F.softmax(y.view(101*257))))
        print(torch.sum(F.softmax(y.view(101*257))))
        
        print("- outputs")
        util.printInfo(outputs)
        util.printInfo(F.softmax(outputs.view(101*257)))
        util.printInfo(F.softmax(F.softmax(outputs.view(101*257))))
        '''
        
        loss = (torch.sum(torch.pow((y -  outputs),2)) / (101*257)) #+ criterion(F.log_softmax(F.softmax(outputs.view(101,257))), F.softmax(F.softmax(y.view(101,257))))
        
        #loss = criterion(F.log_softmax(outputs/torch.sum(outputs),dim=1),F.softmax(y/torch.sum(y),dim=1))
        #loss = torch.sum(torch.pow((y -  outputs),2)) / 8000.0

        loss.backward()
        optimizer.step()
        
        print(loss.data[0])
        
        if idx%20 == 0:
            #util.printInfo(y)
            #util.printInfo(outputs)
            print ('Epoch [%d/%d], Iter [%d/%d], Loss: %.8f'
               %(epoch+1, num_epochs,idx+1, 201, loss.data[0]))
            print("--- %s seconds for epoch ---" % (time.time() - start_time))
            
        lossHistory.append((epoch,idx,loss.data[0]))

    save(outPath, 'epoch' + str(epoch))

save(outPath, 'final')
print(lossHistory)